In [1]:
import os
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import tensorflow as tf
print("TF: ", tf.__version__)
from tensorflow.keras import layers
from tensorflow.keras import models
from keras import Model

from pathlib import Path
from keras.preprocessing.image import load_img, img_to_array, image_dataset_from_directory
from tensorflow.keras.applications import vgg16, vgg19

import wandb
from wandb.keras import WandbCallback

TF:  2.7.0


In [2]:
def seed_everything():
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1' 
    np.random.seed(hash("improves reproducibility") % 2**32 - 1)
    tf.random.set_seed(hash("by removing stochasticity") % 2**32 - 1)

seed_everything()

In [3]:

# Get base project directory
project_path = Path(os.getcwd()).parent.parent
datapath = (project_path /'data/processed/')

CLASSES = {0 : 'yes', 1 : 'no'}
# Loops through pathlist and reads and resizes images
def read_image(pathlist : list, size : int)-> list:
    data = []
    for path in pathlist:
        image=load_img(path, color_mode='rgb', target_size=(size, size))
        image=img_to_array(image)
        # image=image/255.0
        data.append(image)
    return data

# Makes input and label data from folder locations.
# Loops through location "subfolder/CLASSES"
def get_sets(subfolder : str, CLASSES : dict, size : int) -> tuple[list, list]:
    folder_paths = []
    folder_labels = []
    labels = []
    for k,v in CLASSES.items():
        # input datapath generation
        folder_paths += list((datapath / f"2_split_{v}/{subfolder}").rglob("*"))
    # Label data generation
    folder_labels = [0 if x.stem.split('_')[1] == 'yes' else 1 for x in folder_paths]
    # Extract images from datapaths
    img_list = read_image(folder_paths, size)

    return img_list, folder_labels

def get_training_set(CLASSES : dict, size : int) -> tuple[list, list]:
    folder_paths = []
    folder_labels = []
    labels = []
    for k,v in CLASSES.items():
        # input datapath generation
        folder_paths += list((datapath / f"3_aug_{v}_train/").rglob("*"))
        # print(folder_paths)
    # Label data generation
    folder_labels = [0 if x.stem.split('_')[1] == 'yes' else 1 for x in folder_paths]
    # Extract images from datapaths
    img_list = read_image(folder_paths, size)

    return img_list, folder_labels

In [4]:
# Dataset inspect
# Read images to variables
size = 224
# X_train, y_train = get_training_set(CLASSES, size)
X_train, y_train = get_sets('train', CLASSES, size)
X_val, y_val = get_sets('val', CLASSES, size)
X_test, y_test = get_sets('test', CLASSES, size)

In [5]:
#@title
@tf.function
def preprocess(image: tf.Tensor, label: tf.Tensor):
    """
    Preprocess the image tensors and parse the labels
    """
    # Preprocess images
    image = tf.image.convert_image_dtype(image, tf.float32)
    
    # Parse label
    label = tf.cast(label, tf.float32)
    
    return image, label


def prepare_dataloader(images: np.ndarray,
                       labels: np.ndarray,
                       loader_type: str='train',
                       batch_size: int=128):
    """
    Utility function to prepare dataloader.
    """
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))

    if loader_type=='train':
        dataset = dataset.shuffle(1024)

    dataloader = (
        dataset
        .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )

    return dataloader

In [10]:
configs = dict(
    image_width = X_train[0].shape[0],
    image_height = X_train[0].shape[1],
    image_channels = X_train[0].shape[2],
    batch_size = 32,
    class_names = CLASSES,
    model_name = '', # set after model is defined
    pretrain_weights = 'imagenet',
    epochs = 5,
    init_learning_rate = 0.001,
    lr_decay_rate = 0.1,
    optimizer = 'adam',
    loss_fn = 'binary_crossentropy',
    metrics = ['accuracy'],
    earlystopping_patience = 5,
    architecture = "",# To be defined f"{base_model._name.upper()} global_average_pooling2d",
    dataset = "Brain_MRI_Images_for_Brain_Tumor_Detection"
)

In [11]:
trainloader = prepare_dataloader(X_train, y_train, 'train', configs.get('batch_size', 64))
validloader = prepare_dataloader(X_val, y_val, 'valid', configs.get('batch_size', 64))
testloader = prepare_dataloader(X_test, y_test, 'test', configs.get('batch_size', 64))

In [12]:
# train_images, train_labels, valid_images, valid_labels, test_images, test_labels = download_and_prepare_dataset(info)

In [30]:
# For demonstration purposes
log_full = False #@param {type:"boolean"}

if log_full:
    log_train_samples = len(X_train)
else:
    log_train_samples = 5 

print(f'Number of train images : {log_train_samples} to be logged')

Number of train images : 5 to be logged


In [31]:
ds = wandb.Artifact(configs["dataset"], "dataset")

# Initialize an empty table
train_table = wandb.Table(columns=[], data=[])
# Add training data
train_table.add_column('image', X_train[:log_train_samples])
# Add training label_id
train_table.add_column('label_id', y_train[:log_train_samples])
# Add training class names


In [32]:

train_table.add_computed_columns(lambda ndx, row:{
    "images": wandb.Image(row["image"]),
    "class_names": configs['class_names'][str(row["label_id"])]
    })


KeyError: '0'

In [15]:
%%time

# Initialize a new W&B run
run = wandb.init(project="baseline_vgg19",
                 entity="bex_team",
                 group="Pictures",
                 job_type="Baseline")

# Intialize a W&B Artifacts
ds = wandb.Artifact(configs["dataset"], "dataset")

# Initialize an empty table
train_table = wandb.Table(columns=[], data=[])
# Add training data
train_table.add_column('image', X_train[:log_train_samples])
# Add training label_id
train_table.add_column('label_id', y_train[:log_train_samples])
# Add training class names
train_table.add_computed_columns(lambda ndx, row:{
    "images": wandb.Image(row["image"]),
    "class_names": configs['class_names'][str(row["label_id"])]
    })

# Add the table to the Artifact
# ds['train_data'] = train_table

# # Let's do the same for the validation data
# valid_table = wandb.Table(columns=[], data=[])
# valid_table.add_column('image', X_val)
# valid_table.add_column('label_id', y_val)
# valid_table.add_computed_columns(lambda ndx, row:{
#     "images": wandb.Image(row["image"]),
#     "class_name": configs['class_names'][str(row["label_id"])]
#     })
# ds['valid_data'] = valid_table

# Save the dataset as an Artifact
ds.save()

# Finish the run
wandb.finish()

KeyError: '0'

In [33]:
def get_model(input_shape: tuple=(32, 32, 3),
              output_activation: str='sigmoid'):

    inputs = layers.Input(input_shape)

    base_model = vgg19.VGG19(weights=configs['pretrain_weights'], include_top=False, input_shape=input_shape)
    base_model.trainable = False
    configs['architecture'] = f"{base_model._name.upper()} global_average_pooling2d"

    inputs = layers.Input(shape=(X_train[0].shape[0], X_train[0].shape[1], 3))
    x = vgg19.preprocess_input(inputs)
    x = base_model(x)
    x = layers.GlobalAveragePooling2D()(x)
    # x = Flatten()(x)
    outputs = layers.Dense(1, activation=output_activation)(x)

    return models.Model(inputs, outputs, name=f'Baseline_{base_model._name.upper()}')

tf.keras.backend.clear_session()
model = get_model()
model.summary()

Model: "Baseline_VGG19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 vgg19 (Functional)          (None, 1, 1, 512)         20024384  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1)              

In [34]:
checkpoint_filepath = (Path(os.getcwd()) /'model_checkpoint/model_checkpoint')

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [35]:
earlystopper = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=configs['earlystopping_patience'], verbose=0, mode='auto',
    restore_best_weights=True
)

You can use `wandb.log` to log any useful metric/parameter that's not logged by `WandbCallback`. Here we are using a learning rate scheduler to exponentially decay the learning rate after 10 epochs. Notice the use of `wandb.log` to capture the learning rate and `commit=False` in particular.

You can learn more about `wandb.log` [here](https://docs.wandb.ai/guides/track/log).

def lr_scheduler(epoch, lr):
    # log the current learning rate onto W&B
    if wandb.run is None:
        raise wandb.Error("You must call wandb.init() before WandbCallback()")

    wandb.log({'learning_rate': lr}, commit=False)
    
    if epoch < 7:
        return lr
    else:
        return lr * tf.math.exp(-configs['lr_decay_rate'])

lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

In [36]:
def train(config: dict,
          callbacks: list,
          verbose: int=0):
    """
    Utility function to train the model.

    Arguments:
        config (dict): Dictionary of hyperparameters.
        callbacks (list): List of callbacks passed to `model.fit`.
        verbose (int): 0 for silent and 1 for progress bar.
    """

    # Initalize model
    tf.keras.backend.clear_session()
    model = get_model(input_shape=(config.image_width, config.image_height, config.image_channels))
    config['model_name'] = model.name # set


    # Compile the model
    opt = tf.keras.optimizers.Adam(learning_rate=config.init_learning_rate)
    model.compile(optimizer=opt,
                  loss=config.loss_fn,
                  metrics=config.metrics)
# model.compile(loss="binary_crossentropy", optimizer=opt, metrics=metrics)
    # Train the model
    # _ = model.fit(np.array(X_train), np.array(y_train),
    #               epochs=config.epochs,
    #               validation_data=(np.array(X_val),np.array(y_val)),
    #               callbacks=callbacks,
    #               batch_size=config.batch_size,
    #               verbose=verbose)
    _ = model.fit(trainloader,
                  epochs=config.epochs,
                  validation_data=validloader,
                  callbacks=callbacks,
                #   batch_size=config.batch_size,
                  verbose=verbose)
    return model

In [37]:
# Initialize the W&B run
run = wandb.init(project='test_wandb', config=configs, job_type='Baseline')
config = wandb.config

# Define WandbCallback for experiment tracking
wandb_callback = WandbCallback(monitor='val_loss',
                               log_weights=True,
                               log_evaluation=True,
                               validation_steps=5,
                               )
# WandbCallback(data_type='image', training_data=(np.array(X_val),np.array(y_val)), labels=CLASSES, save_model=True, save_graph=True)
# callbacks
callbacks = [earlystopper, wandb_callback]#lr_callback

# Train
model = train(config, callbacks=callbacks, verbose=2)

# Evaluate the trained model
loss, acc = model.evaluate(validloader)
wandb.log({'evaluate/accuracy': acc})

# Close the W&B run.
wandb.finish()

wandb: WARNING Error initializing ValidationDataLogger in WandbCallback. Skipping logging validation data. Error: 


Epoch 1/5
4/4 - 44s - loss: 3.1630 - accuracy: 0.4000 - val_loss: 1.4555 - val_accuracy: 0.5476 - 44s/epoch - 11s/step
Epoch 2/5
4/4 - 40s - loss: 1.3270 - accuracy: 0.5440 - val_loss: 0.5949 - val_accuracy: 0.7381 - 40s/epoch - 10s/step
Epoch 3/5
4/4 - 42s - loss: 0.6289 - accuracy: 0.7120 - val_loss: 0.7999 - val_accuracy: 0.6667 - 42s/epoch - 11s/step
Epoch 4/5
4/4 - 44s - loss: 0.7917 - accuracy: 0.7120 - val_loss: 1.0136 - val_accuracy: 0.6667 - 44s/epoch - 11s/step
Epoch 5/5
4/4 - 41s - loss: 0.8755 - accuracy: 0.6880 - val_loss: 0.8912 - val_accuracy: 0.6905 - 41s/epoch - 10s/step
2/2 [==============================] - 10s 2s/step - loss: 0.8912 - accuracy: 0.6905


accuracy,▁▄██▇
epoch,▁▃▅▆█
evaluate/accuracy,▁
loss,█▃▁▁▂
val_accuracy,▁█▅▅▆
val_loss,█▁▃▄▃
accuracy,0.688
best_epoch,1
best_val_loss,0.59492
epoch,4
evaluate/accuracy,0.69048


In [40]:
#@title
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

def create_gradcam(image, model, last_conv_layer_name, pred_index=None):
    # Preprocess the image array
    image, _ = preprocess(tf.expand_dims(image, axis=0), 0)
    # Get GradCAM
    heatmap = make_gradcam_heatmap(image, model, last_conv_layer_name, pred_index)
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]
    jet_heatmap = tf.image.resize(jet_heatmap, size=(28,28))

    # Overlay
    superimposed_img = jet_heatmap * 0.4 + tf.squeeze(image, axis=0)
    superimposed_img = tf.clip_by_value(superimposed_img, 0.0, 1.0)

    return superimposed_img

In [41]:
last_conv_layer_name = 'block4_conv3'

In [42]:
def validation_processor(ndx, row):
    return {
        "input:image": wandb.Image(row["input"]),
        "target:class": class_table.index_ref(row["target"])
    }

def prediction_processor(ndx, row):
    # Get the validation image
    valid_image = np.array(row["val_row"].get_row()["input:image"].image)

    return {
        "output:class": class_table.index_ref(np.argmax(row["output"])),
        "gradcam": wandb.Image(create_gradcam(valid_image, model, last_conv_layer_name)),
        "output:logits": {class_name: value for (class_name, value) in zip(list(config.class_names.values()), row["output"].tolist())}
    }

In [43]:
# Initialize the W&B run
run = wandb.init(project='baseline_vgg19', config=configs, job_type='train')
config = wandb.config

# Get validation table
data_art = run.use_artifact(f'{configs["dataset"]}:latest', type='dataset')
valid_table = data_art.get("valid_data")

# Create a class table
class_table = wandb.Table(columns=[], data=[])
class_table.add_column("class_name", list(config.class_names.values()))

# Define WandbCallback for experiment tracking
wandb_callback = WandbCallback(
                    log_evaluation=True,
                    validation_row_processor=lambda ndx, row: validation_processor(ndx, row),
                    prediction_row_processor=lambda ndx, row: prediction_processor(ndx, row),
                    validation_steps=4,
                    save_model=False
                )

# callbacks
callbacks = [earlystopper, wandb_callback]#lr_callback

# Train
model = train(config, callbacks=callbacks, verbose=2)

# Evaluate the trained model
loss, acc = model.evaluate(validloader)
wandb.log({'evaluate/accuracy': acc})

# Close the W&B run.
wandb.finish()

wandb: WARNING Error initializing ValidationDataLogger in WandbCallback. Skipping logging validation data. Error: 


Epoch 1/5
4/4 - 43s - loss: 1.2269 - accuracy: 0.4560 - val_loss: 1.3816 - val_accuracy: 0.5238 - 43s/epoch - 11s/step
Epoch 2/5
4/4 - 43s - loss: 1.0762 - accuracy: 0.6320 - val_loss: 1.1772 - val_accuracy: 0.5476 - 43s/epoch - 11s/step
Epoch 3/5
4/4 - 41s - loss: 0.9123 - accuracy: 0.6000 - val_loss: 0.9648 - val_accuracy: 0.4762 - 41s/epoch - 10s/step
Epoch 4/5
4/4 - 40s - loss: 0.7945 - accuracy: 0.6320 - val_loss: 0.8947 - val_accuracy: 0.5476 - 40s/epoch - 10s/step
Epoch 5/5
4/4 - 41s - loss: 0.6999 - accuracy: 0.6720 - val_loss: 0.8413 - val_accuracy: 0.5952 - 41s/epoch - 10s/step
2/2 [==============================] - 13s 3s/step - loss: 0.8413 - accuracy: 0.5952


accuracy,▁▇▆▇█
epoch,▁▃▅▆█
evaluate/accuracy,▁
loss,█▆▄▂▁
val_accuracy,▄▅▁▅█
val_loss,█▅▃▂▁
accuracy,0.672
best_epoch,4
best_val_loss,0.8413
epoch,4
evaluate/accuracy,0.59524
